In [11]:
# requires csg2step and OpenSCAD
# https://github.com/gega/csg2stp

In [1]:
import numpy as np, pandas as pd, shapely
import design_parameters as dp
import os
beampipe0=dp.beampipe0

In [2]:

def mkdir(path):
    # Check whether the specified path exists or not
    isExist = os.path.exists(path)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(path)
mkdir("csg")
mkdir("openscad")

In [3]:
from design_parameters import components

In [32]:
#first create the generic layers:
cm=10
det_height=dp.det_height
det_width=dp.det_width
det_gap=dp.det_gap
for compname in "absorber cover foil1 foil2 pcb insulator".split():
    comp = components[compname]
    scad_file=f"openscad/all_layers_{compname}.scad"
    out_file=f"all_layers_{compname}.csg"
    with open(scad_file, "w") as f:
        print("include <generic_layer.scad>",file=f)
        
        print("union(){", file=f)
        nlayers=54+(compname=='absorber')
        for layer in range(nlayers):
            z=beampipe0.getZ(layer)+comp.z_offset
            holeR=beampipe0.holeR(layer)
            holeX=beampipe0.holeX(layer)
            thickness=comp.thickness
            print(f"generic_layer({z*cm}, {holeX*cm}, {holeR*cm}, {thickness*cm}, det_height={det_height*cm},"+ \
                  f"det_width={det_width*cm}, det_gap={det_gap*cm});", file=f)
            
        print("}", file=f)
    os.system(f'OpenSCAD  -o {out_file} {scad_file}') 
    
    #os.system(f'mv {csg_file} csg/{csg_file}')    
    
        
    

In [37]:


#next create the scintillator cells (based on the database of all layers)
#first read in the database of scintillator positions/shapes
import pickle
with open("all_layers.pkl", "rb") as file:
    all_layers=pickle.load(file)
#print(all_layers)

for r in [(0,9), (9, 18), (18, 54)]:
    scad_file=f"openscad/layers_{r[0]}-{r[1]-1}_scint.scad"
    out_file=f"layers_{r[0]}-{r[1]-1}_scint.csg"
    with open(scad_file, "w") as f:

        print("union(){", file=f)
        #print("include <scint_lib.scad>",file=f)
        for layer in range(r[0], r[1]):

            for side in "LR":
                print(f"// begin layer {layer}, side {side}", file=f)
                df=all_layers[(layer, side)]
                z=beampipe0.getZ(layer)+components['scint'].z_offset

                thickness=components['scint'].thickness
                for cell in range(len(df.scint_boundsx)):
                    if df.deadzone[cell]:
                        continue
                    x,y=df.scint_boundsx[cell], df.scint_boundsy[cell]

                    xystr="[" + (", ".join(['[%.2f, %.2f]'%(x[i]*cm, y[i]*cm) for i in range(len(x))])) + "]"

                    #print(xstr)
                    command=f"translate([0,0,{z*cm}]) linear_extrude({thickness*cm}) polygon(points={xystr});"

                    print(command, file=f)
                print(f"// end layer {layer}, side {side}", file=f)

        print("}", file=f)      
    os.system(f'OpenSCAD  -o {out_file} {scad_file}') 


#note, to convert this to stp in FreeCAD, use the following command in the console to make it easier for it to load:
#  FreeCAD.loadFile(iname)

In [38]:
def frame_str(df, wall_thickness=dp.wall_thickness, frame_height=0.3):
    cm=10
    segments=[]
    ret="frame(segments=["
    lines=[]
    #keep track of 90 degree corners, so that they can be made 90 degree square corners instead of rounded
    corners=[]
    for i in range(len(df)):
        for j in range(len(df.boundsx[i])-1):
            segment = {(df.boundsx[i][j], df.boundsy[i][j]), (df.boundsx[i][j+1], df.boundsy[i][j+1])}
            tol=1e-6
            if j!=0 and (abs(df.boundsx[i][j]-df.boundsx[i][j+1])<tol and abs(df.boundsy[i][j]-df.boundsy[i][j-1])<tol \
                      or abs(df.boundsy[i][j]-df.boundsy[i][j+1])<tol and abs(df.boundsx[i][j]-df.boundsx[i][j-1])<tol) or \
                j==0 and (abs(df.boundsx[i][j]-df.boundsx[i][j+1])<tol and abs(df.boundsy[i][j]-df.boundsy[i][-2])<tol \
                       or abs(df.boundsy[i][j]-df.boundsy[i][j+1])<tol and abs(df.boundsx[i][j]-df.boundsx[i][-2])<tol):
                corner = (df.boundsx[i][j], df.boundsy[i][j])
                #print(corner)
                if corner not in corners:
                    corners.append(corner)
            if segment not in segments:
                lines.append(f"[[{cm*df.boundsx[i][j]:.3f},{cm*df.boundsy[i][j]:.3f}], [{cm*df.boundsx[i][j+1]:.3f}, {cm*df.boundsy[i][j+1]:.3f}]]")
                segments.append(segment)
        #print()
    ret+=",\n".join(lines)+"],"
    
    #add points for square corners if applicable.  
    if corners:
        ret+="corners=["
        lines=[]
        for corner in corners:
            lines.append(f"  [{cm*corner[0]:.3f}, {cm*corner[1]:.3f}]")
        ret+=",\n".join(lines)+"\n],"
    else:
        ret+="corners=[],"
    ret+=f"wall_thickness={wall_thickness*cm},\nframe_height={frame_height*cm});"
    return ret

In [39]:
prefix="""
module connect(start, end, wall_thickness, frame_height){
    dx=start-end;
    length=norm(dx);
    phi=atan2(dx[1],dx[0]);
     translate((start+end)/2) rotate(phi) square([length, wall_thickness],true);
     $fn=12;
     translate(start) circle(r=wall_thickness/2);
     translate(start) circle(r=wall_thickness/2);
}
module frame(segments,corners, wall_thickness=0.8, frame_height=3){
    linear_extrude(frame_height) {
        union(){
            for(segment = segments)
                connect(segment[0],segment[1], wall_thickness, frame_height);
            for (corner =corners)
                translate(corner) square(wall_thickness,center=true);
            }
    }
}
"""
for r in [(0,9), (9, 18), (18, 54)]:
    scad_file=f"openscad/layers_{r[0]}-{r[1]-1}_3dframe.scad"
    out_file=f"layers_{r[0]}-{r[1]-1}_3dframe.csg"
    with open(scad_file, "w") as f:
        print(prefix, file=f)
        for layer in range(*r):
            z = dp.beampipe0.getZ(layer)
            for side in "LR":
                df = all_layers[(layer, side)]
                print(f"translate([0, 0, {z*cm}]) {frame_str(df)}", file=f)

    os.system(f'OpenSCAD -o {out_file} {scadfile}') 